# Tutorial 6: Multiprocessing

In [1]:
import time
import random
import yaml
import os
from pprint import pprint

from theflow import Function, Param, Node, load
from theflow.utils.multiprocess import parallel

class Sum1(Function):
    a: int

    def run(self, x, y, w, task_number) -> int:
        time.sleep(w)
        print(os.getpid(), self.fl.run_id, self.fl.prefix, self.fl.name, task_number)
        return self.a + x + y

class Plus(Function):
    a: int
    func: Function = Node(default=Sum1.withx(a=1))

    def run(self, x, n_times, n_processes) -> int:
        print(self._ff_childs_called)
        tasks = [{"x": x, "y": self.a, "w": random.random() * 5, "task_number":_} for _ in range(n_times)]
        results = list(parallel(self, "func", tasks, processes=n_processes))
        return sum(results)

In [2]:
pip = Plus(a=1, func=Sum1(a=1))
output = pip(1, 10, 2)

{}
335337 17030115459146364 . func 0
335337 17030115459146364 . func[2] 2
335341 17030115459146364 . func[1] 1
335337 17030115459146364 . func[3] 3
335341 17030115459146364 . func[4] 4
335337 17030115459146364 . func[5] 5
335341 17030115459146364 . func[6] 6
335337 17030115459146364 . func[7] 7
335337 17030115459146364 . func[9] 9
335341 17030115459146364 . func[8] 8


In [3]:
pip.context.get_all_contexts()

{'__main__.Plus|17030115459146364': {'run_id': '17030115459146364'},
 '__main__.Plus|17030115459146364|.': {},
 '__main__.Plus|17030115459146364|__progress__': {'name': '__main__.Plus',
  'id': '17030115459146364',
  '.': {'status': 'run',
   'input': {'args': (1, 10, 2), 'kwargs': {}},
   'output': 30},
  '.func': {'status': 'run',
   'input': {'args': (),
    'kwargs': {'x': 1, 'y': 1, 'w': 0.44078312844893, 'task_number': 0}},
   'output': 3},
  '.func[1]': {'status': 'run',
   'input': {'args': (),
    'kwargs': {'x': 1, 'y': 1, 'w': 2.6881405205439393, 'task_number': 1}},
   'output': 3},
  '.func[2]': {'status': 'run',
   'input': {'args': (),
    'kwargs': {'x': 1, 'y': 1, 'w': 1.372465578250785, 'task_number': 2}},
   'output': 3},
  '.func[3]': {'status': 'run',
   'input': {'args': (),
    'kwargs': {'x': 1, 'y': 1, 'w': 1.1409940026848187, 'task_number': 3}},
   'output': 3},
  '.func[4]': {'status': 'run',
   'input': {'args': (),
    'kwargs': {'x': 1, 'y': 1, 'w': 3.39485

-------